In [ ]:
import pygame
import math
import os
import threading
import time
import random
import cv2  # For webcam and drawing
import mediapipe as mp # For hand tracking

# --- Pygame Initialization ---
pygame.init()

# Screen dimensions
SCREEN_WIDTH = 1280
SCREEN_HEIGHT = 800
SCREEN = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Gesture-Controlled Drifting Car")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

In [10]:
try:
    CAR_IMAGE_ORIGINAL = pygame.image.load(os.path.join('assets', 'car.png')).convert_alpha()
    road_images_paths = ['road_1.png', 'road_2.png', 'road_3.png']
    ROAD_IMAGES = [
        pygame.transform.scale(
            pygame.image.load(os.path.join('assets', path)).convert_alpha(),
            (SCREEN_WIDTH, SCREEN_HEIGHT)
        ) for path in road_images_paths
    ]
    # All traffic car images are now loaded here
    traffic_car_images = [
        pygame.transform.scale(pygame.image.load(os.path.join('assets', 'traffic_car_1.png')).convert_alpha(), (50, 100)),
        pygame.transform.scale(pygame.image.load(os.path.join('assets', 'traffic_car_2.png')).convert_alpha(), (50, 100)),
        pygame.transform.scale(pygame.image.load(os.path.join('assets', 'traffic_car_3.png')).convert_alpha(), (50, 100)),
        pygame.transform.scale(pygame.image.load(os.path.join('assets', 'traffic_car_4.png')).convert_alpha(), (50, 100))
    ]
    CAR_IMAGE_ORIGINAL = pygame.transform.scale(CAR_IMAGE_ORIGINAL, (50, 100))
except pygame.error as e:
    print(f"Error loading assets: {e}")
    print("Please make sure your 'assets' folder is set up correctly.")
    pygame.quit()

# Game clock
CLOCK = pygame.time.Clock()
FPS = 60

In [11]:
class Car(pygame.sprite.Sprite):
    def __init__(self, image, x, y):
        super().__init__()
        self.original_image = image
        self.image = self.original_image
        self.rect = self.image.get_rect(center=(x, y))
        self.pos = pygame.math.Vector2(x, y)
        self.speed = 0
        self.acceleration_rate = 0.5
        self.max_speed = 100
        self.reverse_speed = 2
        self.horizontal_speed = 6
        self.dodge_speed_multiplier = 1.8
        self.mask = pygame.mask.from_surface(self.image)
        self.is_dodging = False

    def update(self, current_gesture, game_state):
        if game_state != GameState.PLAYING:
            self.speed = 0
            return

        # Use 'both fists' gesture as a handbrake during gameplay
        if current_gesture == 'handbrake_calibrate':
            self.speed *= 0.85 # Stronger braking
        else:
            self.speed *= 0.98 # Normal friction
            
        if current_gesture == 'accelerate':
            self.speed += self.acceleration_rate
        
        self.speed = max(-self.reverse_speed, min(self.speed, self.max_speed))
        if abs(self.speed) < 0.1: self.speed = 0

        applied_horizontal_speed = self.horizontal_speed
        self.is_dodging = False

        if current_gesture in ['left_ok_sign', 'right_ok_sign']:
            applied_horizontal_speed *= self.dodge_speed_multiplier
            self.is_dodging = True
            if current_gesture == 'left_ok_sign': self.pos.x -= applied_horizontal_speed
            else: self.pos.x += applied_horizontal_speed
        elif current_gesture in ['left_turn', 'right_turn']:
            if current_gesture == 'left_turn': self.pos.x -= applied_horizontal_speed
            else: self.pos.x += applied_horizontal_speed

        road_left_edge, road_right_edge = SCREEN_WIDTH * 0.35, SCREEN_WIDTH * 0.65
        half_car_width = self.rect.width / 2
        self.pos.x = max(road_left_edge + half_car_width, min(self.pos.x, road_right_edge - half_car_width))
        self.rect.center = (int(self.pos.x), int(self.pos.y))

class TrafficCar(pygame.sprite.Sprite):
    def __init__(self, image_list, x, y, base_speed_increase=0):
        super().__init__()
        self.image = random.choice(image_list)
        self.rect = self.image.get_rect(center=(x, y))
        self.speed = random.uniform(2, 4) + base_speed_increase
        self.mask = pygame.mask.from_surface(self.image)

    def update(self, road_scroll_speed):
        self.rect.y += road_scroll_speed + self.speed
        if self.rect.top > SCREEN_HEIGHT:
            self.kill()

class GameState:
    CALIBRATING = 0
    READY = 1
    PLAYING = 2
    GAME_OVER = 3

In [12]:
HIGHSCORE_FILE = "highscore.txt"

def save_highscore(score):
    try:
        with open(HIGHSCORE_FILE, "w") as f:
            f.write(str(int(score)))
    except IOError:
        print(f"Error: Unable to save high score to {HIGHSCORE_FILE}")

def load_highscore():
    try:
        with open(HIGHSCORE_FILE, "r") as f:
            return int(f.read())
    except (IOError, ValueError):
        return 0

def draw_text(surface, text, size, color, x, y, center=True):
    font = pygame.font.Font(None, size)
    text_surface = font.render(text, True, color)
    text_rect = text_surface.get_rect()
    if center:
        text_rect.center = (x, y)
    else:
        text_rect.topleft = (x, y)
    surface.blit(text_surface, text_rect)

In [13]:
# --- High Score and Asset Loading ---

HIGHSCORE_FILE = "highscore.txt"

def save_highscore(score):
    """Saves the high score to a file."""
    try:
        with open(HIGHSCORE_FILE, "w") as f:
            f.write(str(int(score)))
    except IOError:
        print(f"Error: Unable to save high score to {HIGHSCORE_FILE}")

def load_highscore():
    """Loads the high score from a file."""
    try:
        with open(HIGHSCORE_FILE, "r") as f:
            return int(f.read())
    except (IOError, ValueError):
        # Return 0 if the file doesn't exist or contains invalid data
        return 0

# This list was missing, which would cause the next crash.
# It's better to define it here, outside the game loop.
try:
    traffic_car_images = [
        pygame.transform.scale(pygame.image.load(os.path.join('assets', 'traffic_car_1.png')).convert_alpha(), (50, 100)),
        pygame.transform.scale(pygame.image.load(os.path.join('assets', 'traffic_car_2.png')).convert_alpha(), (50, 100)),
        pygame.transform.scale(pygame.image.load(os.path.join('assets', 'traffic_car_3.png')).convert_alpha(), (50, 100)),
        pygame.transform.scale(pygame.image.load(os.path.join('assets', 'traffic_car_4.png')).convert_alpha(), (50, 100))
        # You can add more traffic car images here, e.g., 'traffic_car_2.png'
    ]
except pygame.error as e:
    print(f"Error loading traffic car assets: {e}")
    pygame.quit()
    exit()

In [14]:
# This dictionary is updated by the hand tracking thread and read by the game loop.
shared_gesture_data = {'gesture': None, 'confidence': 0.0}
is_game_running = True # A flag to signal the thread to stop

def hand_tracker_thread():
    global shared_gesture_data, is_game_running
    
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    hands = mp_hands.Hands(
        max_num_hands=2, # We need to detect two hands
        min_detection_confidence=0.7,
        min_tracking_confidence=0.7
    )
    
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    def get_hand_state(hand_landmarks):
        """Classifies a single hand as FIST, OPEN, or OK_SIGN."""
        landmarks = hand_landmarks.landmark
        
        # Helper to check if a finger is extended vertically
        def is_finger_up(tip_idx, dip_idx):
            return landmarks[tip_idx].y < landmarks[dip_idx].y

        # Check for open palm (all 4 fingers up)
        if all(is_finger_up(tip, dip) for tip, dip in [(8,6), (12,10), (16,14), (20,18)]):
            return 'OPEN'
        
        # Check for OK sign (thumb and index tips are close)
        thumb_tip = landmarks[mp_hands.HandLandmark.THUMB_TIP]
        index_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP]
        if math.hypot(thumb_tip.x - index_tip.x, thumb_tip.y - index_tip.y) < 0.07:
            return 'OK_SIGN'

        # If not open and not OK, assume it's a fist
        return 'FIST'

    while cap.isOpened() and is_game_running:
        success, image = cap.read()
        if not success: continue

        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        left_hand_state, right_hand_state = None, None
        final_gesture = None
        
        if results.multi_hand_landmarks:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                hand_label = handedness.classification[0].label
                state = get_hand_state(hand_landmarks)
                if hand_label == "Left": left_hand_state = state
                else: right_hand_state = state

            # --- Gesture Combination Logic (with priority) ---
            # Highest priority: Drifting
            if left_hand_state == 'OK_SIGN': final_gesture = 'left_ok_sign'
            elif right_hand_state == 'OK_SIGN': final_gesture = 'right_ok_sign'
            # Next priority: Turning
            elif left_hand_state == 'OPEN' and right_hand_state == 'FIST': final_gesture = 'left_turn'
            elif left_hand_state == 'FIST' and right_hand_state == 'OPEN': final_gesture = 'right_turn'
            # Next: Acceleration
            elif left_hand_state == 'OPEN' and right_hand_state == 'OPEN': final_gesture = 'accelerate'
            # Last: Calibration / Braking
            elif left_hand_state == 'FIST' and right_hand_state == 'FIST': final_gesture = 'handbrake_calibrate'

        shared_gesture_data = {'gesture': final_gesture, 'confidence': 1.0 if final_gesture else 0.0}

        # Display gesture text on the webcam feed
        if final_gesture:
            cv2.putText(image, final_gesture.upper(), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        cv2.imshow('Gesture Controller', image)
        if cv2.waitKey(5) & 0xFF == 27: # Exit on ESC
            break
            
    hands.close()
    cap.release()
    cv2.destroyAllWindows()

In [15]:
def game_loop():
    global current_game_state, shared_gesture_data

    running = True
    car = None
    traffic_group = pygame.sprite.Group()
    high_score = load_highscore()
    
    camera_angle, target_angle = 0, 0
    game_surface = pygame.Surface((SCREEN_WIDTH, SCREEN_HEIGHT), pygame.SRCALPHA)
    
    def reset_game_state():
        nonlocal car, traffic_group, current_score, difficulty_level, score_for_next_level, base_traffic_speed_increase, road_y_offset, camera_angle, target_angle
        car = Car(CAR_IMAGE_ORIGINAL, SCREEN_WIDTH // 2, SCREEN_HEIGHT - 150)
        traffic_group.empty()
        current_score, difficulty_level = 0, 0
        score_for_next_level, base_traffic_speed_increase = 500, 0
        road_y_offset, camera_angle, target_angle = 0, 0, 0

    reset_game_state()
    current_game_state = GameState.CALIBRATING
    
    calibration_start_time, calibration_duration, calibration_gesture_detected = 0, 3, False
    LANE_CENTERS = [SCREEN_WIDTH * 0.38, SCREEN_WIDTH * 0.44, SCREEN_WIDTH * 0.50, SCREEN_WIDTH * 0.56, SCREEN_WIDTH * 0.62]
    SPAWN_TRAFFIC_EVENT = pygame.USEREVENT + 1

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
                running = False
            
            if event.type == SPAWN_TRAFFIC_EVENT and current_game_state == GameState.PLAYING:
                lane = random.choice(LANE_CENTERS)
                traffic_group.add(TrafficCar(traffic_car_images, lane, -100, base_traffic_speed_increase))
                min_spawn_time = max(200, 700 - difficulty_level * 100)
                max_spawn_time = max(600, 2000 - difficulty_level * 200)
                pygame.time.set_timer(SPAWN_TRAFFIC_EVENT, random.randint(min_spawn_time, max_spawn_time), loops=1)

            if event.type == pygame.KEYDOWN:
                if current_game_state == GameState.GAME_OVER and event.key == pygame.K_c:
                    reset_game_state()
                    current_game_state = GameState.CALIBRATING
                elif event.key == pygame.K_0: running = False

        current_gesture_command = shared_gesture_data.get('gesture')

        if current_game_state == GameState.CALIBRATING:
            SCREEN.fill(BLACK)
            draw_text(SCREEN, "CALIBRATING: Show Both Fists Closed", 40, WHITE, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 - 50)
            if current_gesture_command == 'handbrake_calibrate':
                if not calibration_gesture_detected:
                    calibration_start_time = time.time()
                    calibration_gesture_detected = True
                elapsed_time = time.time() - calibration_start_time
                if elapsed_time >= calibration_duration:
                    current_game_state = GameState.READY
                    calibration_gesture_detected = False
                else:
                    draw_text(SCREEN, f"Calibrating in {int(calibration_duration - elapsed_time) + 1}...", 50, GREEN, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 100)
            else:
                calibration_gesture_detected = False

        elif current_game_state == GameState.READY:
            SCREEN.fill(BLACK)
            draw_text(SCREEN, "READY!", 50, GREEN, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 - 50)
            draw_text(SCREEN, "Show Both Open Palms to Accelerate", 30, WHITE, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 20)
            if current_gesture_command == 'accelerate':
                current_game_state = GameState.PLAYING
                pygame.time.set_timer(SPAWN_TRAFFIC_EVENT, 100, loops=1)

        elif current_game_state == GameState.PLAYING:
            if current_score > score_for_next_level:
                difficulty_level += 1
                score_for_next_level += 500 + (difficulty_level * 200)
                base_traffic_speed_increase += 0.5
            
            road_scroll_speed = max(0, car.speed) / 2
            current_score += car.speed / 10 if car.speed > 0 else 0.1
            
            car.update(current_gesture_command, current_game_state)
            traffic_group.update(road_scroll_speed)
            
            target_angle = 0
            if car.is_dodging:
                if 'left' in current_gesture_command: target_angle = 15
                else: target_angle = -15
            camera_angle += (target_angle - camera_angle) * 0.1

            if pygame.sprite.spritecollide(car, traffic_group, False, pygame.sprite.collide_mask):
                current_game_state = GameState.GAME_OVER
                if current_score > high_score:
                    high_score = current_score
                    save_highscore(high_score)

            game_surface.fill((0,0,0,0))
            level_index = (difficulty_level // 2) % len(ROAD_IMAGES)
            current_road_image = ROAD_IMAGES[level_index]
            road_y_offset = (road_y_offset + road_scroll_speed) % SCREEN_HEIGHT
            game_surface.blit(current_road_image, (0, road_y_offset - SCREEN_HEIGHT))
            game_surface.blit(current_road_image, (0, road_y_offset))
            traffic_group.draw(game_surface)
            game_surface.blit(car.image, car.rect)
            rotated_surface = pygame.transform.rotate(game_surface, camera_angle)
            rotated_rect = rotated_surface.get_rect(center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2))
            SCREEN.fill(BLACK)
            SCREEN.blit(rotated_surface, rotated_rect)
            
            draw_text(SCREEN, f"Score: {int(current_score)}", 30, (200,200,255), SCREEN_WIDTH - 120, 20)
            draw_text(SCREEN, f"High Score: {int(high_score)}", 25, (200,200,255), SCREEN_WIDTH - 120, 60)
            draw_text(SCREEN, f"Level: {difficulty_level}", 25, WHITE, SCREEN_WIDTH - 120, 100)

        elif current_game_state == GameState.GAME_OVER:
            SCREEN.fill(BLACK)
            draw_text(SCREEN, "GAME OVER!", 60, RED, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 - 100)
            draw_text(SCREEN, f"Your Score: {int(current_score)}", 40, WHITE, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 - 20)
            draw_text(SCREEN, "Press 'C' to Recalibrate", 30, GREEN, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 60)
            draw_text(SCREEN, "Press '0' or ESC to Exit", 30, WHITE, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 100)
            
        pygame.display.flip()
        CLOCK.tick(FPS)
    
    global is_game_running
    is_game_running = False
    pygame.quit()

In [16]:
# Create an 'assets' directory if it doesn't exist
if not os.path.exists('assets'):
    os.makedirs('assets')
    print("Created 'assets' directory. Please place image files inside it.")
else:
    # Start the hand tracking thread.
    tracker_thread = threading.Thread(target=hand_tracker_thread, daemon=True)
    tracker_thread.start()
    
    # Start the game
    # Adding a small delay to ensure the camera can initialize
    time.sleep(2) 
    game_loop()